In [1]:
import csv

In [3]:
with open('results_oakland_and_global.csv', 'r') as f:
    reader_list = csv.DictReader(f)
    temperatures = list(reader_list)

In [13]:
#check if the years are consecutive, i.e., all years accounted for from 1849 to 2013
temp_list = [int(temp['year']) for temp in temperatures]
temp_list == list(range(temp_list[0], temp_list[-1]+1)) # temp_list[0]=1849, temp_list[-1]=2013

True

In [14]:
import pandas as pd

In [15]:
data = pd.read_csv('results_oakland_and_global.csv')

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   year             165 non-null    int64  
 1   city             165 non-null    object 
 2   country          165 non-null    object 
 3   avg_temp         165 non-null    float64
 4   global_year      165 non-null    int64  
 5   global_avg_temp  165 non-null    float64
dtypes: float64(2), int64(2), object(2)
memory usage: 7.9+ KB


# it looks like there is data in every column
# there is need to clean data for now